In [1]:
# Importing Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Setting the Path
path = 'CT_images'

In [3]:
# Loading the images
train_data_dir = os.path.join(path, 'Train')
test_data_dir = os.path.join(path, 'Test')

In [4]:
# Image Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Training the Model
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150,150),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

Found 5000 images belonging to 2 classes.
Found 1454 images belonging to 2 classes.


In [6]:
model = Sequential()

#Adding convolutional layers
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Adding a second convolutional layer
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [7]:
# Compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
# Training the Model
model.fit(
    train_generator,
    steps_per_epoch=32,
    epochs=80,
    validation_data=test_generator,
    validation_steps=32)

Epoch 1/80
32/32 [==============================] - 33s 985ms/step - loss: 0.4978 - accuracy: 0.8164 - val_loss: 0.0669 - val_accuracy: 1.0000
Epoch 2/80
32/32 [==============================] - 28s 875ms/step - loss: 0.2756 - accuracy: 0.8930 - val_loss: 0.1790 - val_accuracy: 0.9346
Epoch 3/80
32/32 [==============================] - 28s 875ms/step - loss: 0.1064 - accuracy: 0.9648 - val_loss: 0.8093 - val_accuracy: 0.6836
Epoch 4/80
32/32 [==============================] - 28s 879ms/step - loss: 0.0560 - accuracy: 0.9834 - val_loss: 0.7405 - val_accuracy: 0.7461
Epoch 5/80
32/32 [==============================] - 27s 862ms/step - loss: 0.0291 - accuracy: 0.9890 - val_loss: 0.9687 - val_accuracy: 0.7451
Epoch 6/80
32/32 [==============================] - 28s 874ms/step - loss: 0.0255 - accuracy: 0.9951 - val_loss: 0.9948 - val_accuracy: 0.7520
Epoch 7/80
32/32 [==============================] - 29s 901ms/step - loss: 0.0173 - accuracy: 0.9930 - val_loss: 1.0070 - val_accuracy: 0.7871

Epoch 58/80
32/32 [==============================] - 28s 877ms/step - loss: 0.0047 - accuracy: 0.9990 - val_loss: 3.5776 - val_accuracy: 0.7354
Epoch 59/80
32/32 [==============================] - 28s 877ms/step - loss: 0.0084 - accuracy: 0.9980 - val_loss: 3.6308 - val_accuracy: 0.6455
Epoch 60/80
32/32 [==============================] - 28s 883ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 2.8881 - val_accuracy: 0.6885
Epoch 61/80
32/32 [==============================] - 28s 890ms/step - loss: 0.0074 - accuracy: 0.9990 - val_loss: 4.1582 - val_accuracy: 0.6328
Epoch 62/80
32/32 [==============================] - 28s 876ms/step - loss: 0.0100 - accuracy: 0.9971 - val_loss: 2.1778 - val_accuracy: 0.7207
Epoch 63/80
32/32 [==============================] - 28s 894ms/step - loss: 0.0077 - accuracy: 0.9951 - val_loss: 2.6778 - val_accuracy: 0.6855
Epoch 64/80
32/32 [==============================] - 28s 883ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 3.2554 - val_accuracy:

In [9]:
model.evaluate(test_generator,steps=40)

40/40 [==============================] - 13s 323ms/step - loss: 2.6391 - accuracy: 0.7648


[2.6390764713287354, 0.764843761920929]

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

In [11]:
# Saving the Model
model.save('kidney_stone_detection_model.h5')

In [12]:
model = keras.models.load_model('kidney_stone_detection_model.h5')

In [14]:
# Creating the GUI
from tkinter import *
from PIL import Image, ImageTk

import tkinter
import tkinter as tk
import customtkinter
from tkinter import filedialog
from tkinter.filedialog import askopenfilename
from tkinter import messagebox
import keras.utils as image
import numpy as np
import os

customtkinter.set_appearance_mode("System")
root = customtkinter.CTk()

#window size
root.rowconfigure(0,weight=1)
root.columnconfigure(0,weight=1)

root.geometry('420x270')

root.title('Kidney Stone Detection')

def reset_btn():
    global label_prediction
    result = tk.Label(root, text="                                                  ")
    result.grid(row=1,column=1,sticky='nw',padx=20,pady=20)

# Function to Select Image
def browse_btn():
    global image_name
    image_name = askopenfilename(title='Select Image')
    
    img = Image.open(image_name)
    img = img.resize((200, 200), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    panel = tk.Label(root, image=img)
    panel.image = img
    panel.grid(row=0,column=1,sticky='nw',padx=20,pady=20)

# Function to Predict
def predict_btn():
    global label_prediction
    global image_name
    test_img = image.load_img(image_name, target_size=(150, 150))
    test_img = image.img_to_array(test_img)
    test_img = np.expand_dims(test_img, axis=0)
    result = model.predict(test_img)
    print(result)
    if result[0][0] == 1:
        result = tk.Label(root, text="Kidney Stone Detected")
        result.grid(row=1,column=1,sticky='nw',padx=20,pady=20)
    elif result[0][0] == 0:
        result = tk.Label(root, text="No Kidney Stone Detected")
        result.grid(row=1,column=1,sticky='nw',padx=20,pady=20)

# Browse Button
browsebtn = customtkinter.CTkButton(master=root, text="Browse Image", command=browse_btn)
browsebtn.grid(row=0, column=0,sticky='nw',padx=20,pady=20)

# Predict Butoon
predictbtn = customtkinter.CTkButton(master=root, text="Predict", command=predict_btn)
predictbtn.grid(row=1, column=0,sticky='nw',padx=20,pady=20)
predictbtn1 = customtkinter.CTkButton(master=root, text="Reset", command=reset_btn)
predictbtn1.grid(row=2, column=0,sticky='nw',padx=20,pady=20)




# Running the GUI
root.mainloop()